## Tobigs 4주차 SVM 과제 - Multiclass SVM
#### 15기 이윤정
---
기본적으로 SVM은 binary classification이므로 multi classification을 하기 위해서는 2가지 방법이 존재한다.   
    1. One vs Rest  
    2. One vs One

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
scal = StandardScaler() #scaling
X = scal.fit_transform(X)

In [3]:
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [4]:
# One VS Rest 구현을 위해 1) class가 0 or not 2)class가 1 or not을 구분하기 위한 머신 등을 미리 만들어주자
svm_1 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_2 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_3 = SVC(kernel ='rbf', C = 5, gamma = 5)

In [5]:
from sklearn.model_selection import train_test_split #test/train 데이터로 분리

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [6]:
y_train = pd.get_dummies(y_train) #one hot encoding

In [7]:
svm_1.fit(X_train,y_train.iloc[:,0]) # 데이터 클레스가 0 or not 구분해주는 머신
svm_2.fit(X_train,y_train.iloc[:,1]) # 데이터 클레스가 1 or not 구분해주는 머신
svm_3.fit(X_train,y_train.iloc[:,2]) # 데이터 클레스가 2 or not 구분해주는 머신
print(svm_1.predict(X_test)) #데이터 클래스가 0 or not을 구분해주는 애를 통해서 테스트 데이터의 클래스가 0인지, 0이 아닌인지 예측해보자

print(svm_1.decision_function(X_test)) #decision_function hyperplane과의 거리를 구하는 방법(필요하다면 활용해주세요!)

[0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]
[-1.12470845 -0.86326953 -0.65281099 -0.50248821 -0.76284323 -0.87465573
  1.07709345 -0.99281647  0.47441336 -0.99842743 -0.83989348  0.15633457
  0.32871788 -0.97965464 -0.72385083 -0.92638376  1.28322481 -0.56240455
 -0.72719892 -0.99509775  0.43166724 -0.96451557 -0.82991366 -1.03020581
 -0.75166835  1.13461335  0.39943974 -1.04194106 -0.93376548 -1.06133798]


In [10]:
# 부호가 모든 같은 경우가 있는가? > 모두 동점인 경우가 생길 것
for i in range(len(X_test)):
    # ~. decision_function을 이용하면 해당 데이터가 하이퍼플레인으로부터 얼마나 떨어져있는지 '거리'가 나온다!
    # 다음은 그 값의 부호를 이용해 모두가 동점인 경우가 있는지 출력하는 함수 
    if (np.sign(svm_1.decision_function(X_test)[i]) == np.sign(svm_2.decision_function(X_test)[i])) and (np.sign(svm_2.decision_function(X_test)[i]) == np.sign(svm_3.decision_function(X_test)[i])):
        print(i)

3
17
18


---
## One vs Rest(All) SVM
: 전체 class의 수가 M개 일때, 각 class 별로 해당 class(1) vs 이외의 class(m-1)로 binary decision을 한 다음 Hypothesis function 값이 가장 큰 값을 고르는 방법  
이때, Hypothesis function 값은 주어진 x가 해당 class에 들어갈 확률이므로 Hypothesis function 값이 크다는 것은 해당 class에 들어갈 확률이 가장 크다는 것을 의미한다.  

**문제점**  
    1. Hypothesis function value = 초평면값의 크기를 단순 비교하여 값을 찾는다는 점이 비합리적일 수 있다.   
    2. binary classifier의 training set이 불균형하다. (1 : m-1)

In [11]:
## Case 1 : One vs Rest SVM을 이부분에 구현해주세요 위 결과들을 이용해서 multi class SVM을 직접 구현해주세요! 하드코딩이 하시기 편할겁니다.

#해당 class vs 이외 class 간의 결과를 기반으로 한 투표 진행
array = [[0 for j in range(3)] for i in range(len(X_test))] #svm 결과 저장할 2차원 배열 선언 및 초기화
y_pred = []

for i in range(0, len(X_test)):  #binary classification SVM 시 자신의 class로 분류한 경우 1로 대입
    if svm_1.predict(X_test)[i] == 1:
        array[i][0] = 1
    elif svm_2.predict(X_test)[i] == 1:
        array[i][1] = 1
    elif svm_3.predict(X_test)[i] == 1:
        array[i][2] = 1
    
    #동점자 처리 - decision_function 이용
    ## case1 - 모두 자신의 class가 아니라고 분류한 경우 = All 0 
    if (array[i][0] == 0) and (array[i][1] == 0) and (array[i][2] == 0):
        y_pred.append(np.argmax([svm_1.decision_function(X_test)[i], svm_2.decision_function(X_test)[i], svm_3.decision_function(X_test)[i]]))
        
    ## case2 - 2개 이상의 class에서 자신의 class라고 분류한 경우
    elif (array[i][0] == 1) and (array[i][1] == 1):
        y_pred.append(np.argmax([svm_1.decision_function(X_test)[i], svm_2.decision_function(X_test)[i]]))  
    elif (array[i][0] == 1) and (array[i][2] == 1):
        y_pred.append(np.argmax([svm_1.decision_function(X_test)[i], svm_3.decision_function(X_test)[i]]))
    elif (array[i][1] == 1) and (array[i][2] == 1):
        y_pred.append(np.argmax([svm_2.decision_function(X_test)[i], svm_3.decision_function(X_test)[i]]))
    
    ## 그외    
    else:
        y_pred.append(np.argmax(array[i]))  #가장 큰 값을 예측값으로 선정

label = []
for j in range(0, len(y_pred)):  #labeling
    if y_pred[j] == 0:
        label.append('setosa')
    elif y_pred[j] == 1:
        label.append('versicolor')
    elif y_pred[j] == 2:
        label.append('virginica')
        
print(label)
print('Accuacy : {: .5f}'.format(accuracy_score(y_test, label)))

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']
Accuacy :  0.86667


---
## One vs One SVM
: 전체 class의 수가 M개 일때, 모든 Class에 대해 1 vs 1로 SVM을 여러개 만들고 확장하는 방식. M개의 클래스 중 2개를 선택하여 두 Class에 대한 결정 초평면을 만든다. (이때, 결정 초평면은 mC2개)   
이후, 새로 들어온 sample에 대해 결정 초평면이 class A로 분류하는 경우 class A에 +1, class B로 분류하는 경우 class B에 +1로 투표한다. 이렇게 모든 결정 초평면에 대해 분류하였을 때, 가장 높은 점수를 획득한 class가 예측된 class이다. (이때, 얻을 수 있는 최대 표 값은 M-1개)

**문제점**  
- SVM의 개수가 기하급수적으로 증가한다.   
- SVM의 개수가 증가함에 따라 learning과 test에 걸리는 시간이 증가한다.
---

In [16]:
# 원래 라이브러리가 제공하는 multi class SVM과 여러분이 구현한 multiclass SVM 결과를 비교해주세요
from sklearn.model_selection import train_test_split #데이터셋 분리
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=48)

svm_4 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_4.fit(X_train_2, y_train_2)
y_pred = svm_4.predict(X_test_2)

# metrics.accuracy_score(y_test_2,y_pred)
accuracy_score(y_test_2,y_pred)

0.8666666666666667

One vs Rest SVM으로 직접 구현하였을 때, Accuacy score = 0.86667이다. 이때, sklearn에서 제공하는 multiclass svm의 Accuacy score = 0.8666666666666667으로 동일한 결과가 도출되었다. 그러므로, 직접 구현한 One vs Rest SVM은 정상 작동된다고 볼 수 있다. 